# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Carregando a base de dados com as mensagens dos seus arquivos:

In [ ]:
train = pd.read_csv('dados_treino_QUARTETO_Thomas.csv')
train.head(5)

In [ ]:
test = pd.read_csv('dados_teste_QUARTETO_Thomas.csv')
test.head(5)

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
# Limpando a base de dados

import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$''()]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [ ]:
# Função que recebe um dataframe, limpa e transforma em pd.Series

def transf_lista(dataframe):
    texto = " "

    for linha in dataframe['Review']:
        texto = texto + cleanup(linha)
    
    lista = texto.lower().split()
    pdseries = pd.Series(lista)
    return pdseries

In [ ]:
# Criando pd.Series para cada classificação distinta e para o total

serie_train = transf_lista(train)
serie_Detractor = transf_lista(test.loc[(train.NPS == 'Detractor'), :])
serie_Promoter = transf_lista(test.loc[(train.NPS == 'Promoter'), :])
serie_Passive = transf_lista(test.loc[(train.NPS == 'Passive'), :])

serie_Detractor

In [ ]:
# Criando uma tabela com a contagem de cada palavra em cada classificação

tabela_Detractor = serie_Detractor.value_counts()
tabela_Promoter = serie_Promoter.value_counts()
tabela_Passive = serie_Passive.value_counts()

tabela_Detractor

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens. Ex: stemming, lemmatization, stopwords
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**